## 卷积运算
卷积：一个卷积核函数在输入信号序列化的积分运算

卷积核不断的移动输出，得到特征图的像素，直到覆盖原始图像

假设原始图像的尺寸为n，卷积核的宽度是w，特征图的大小一般为(n-w+1)*(n-w+1)；若不想让图像变小，就需要使用padding技术将原始图放大，并用0来填补补充区域

有多少个卷积核，就有多少个特征图，不同的特征图结合在一起形成立方体

## 池化运算
获取粗粒度信息，忽略图像中的细节信息

该运算的区域**不重叠**

## 立体卷积核
将每一个长方体中的像素与相应的连接权重相乘，然后再把所有的乘积相加，得到输出特征图上一点的像素值

输出特征图有多少层，这一层卷积就有多少卷积核

对于原始输入图像有多个通道的情况，将其等同于输入为长方体的情况

# 手写数字识别器


In [26]:
# pip install torchvision

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import torchvision.datasets as dsets
import torchvision.transforms as transforms

import matplotlib.pyplot as plt
import numpy as np

In [3]:
matplotlib inline

In [4]:
# 定义超参数
image_size=28
num_classes=10
num_epochs=20
batch_size=64

## 加载数据集
MNIST数据为torchvision包自带数据

想要调用自己的数据，可以用torchvision.datasets.ImageFolder或torch.utils.data.TensorDataset

In [5]:
train_dataset=dsets.MNIST(root='./data',
                         train=True,
                         transform=transforms.ToTensor(),
                         download=True)
test_dataset=dsets.MNIST(root='./data',
                        train=False,
                        transform=transforms.ToTensor())
#训练集的加载器，自动将数据切分成批，顺序随机打乱
train_loader=torch.utils.data.DataLoader(dataset=train_dataset,
                                        batch_size=batch_size,
                                        shuffle=True)
#定义校验数据与测试数据
indices=range(len(test_dataset))
indices_val=indices[:5000]
indices_test=indices[5000:]
#根据下标构建采样器
sampler_val=torch.utils.data.sampler.SubsetRandomSampler(indices_val)
sampler_test=torch.utils.data.sampler.SubsetRandomSampler(indices_test)
#根据采样器定义加载器
validation_loader=torch.utils.data.DataLoader(dataset=test_dataset,
                                             batch_size=batch_size,
                                             shuffle=False,
                                             sampler=sampler_val)
test_loader=torch.utils.data.DataLoader(dataset=test_dataset,
                                       batch_size=batch_size,
                                       shuffle=False,
                                       sampler=sampler_test)

### torch.utils.data 管理数据集的工具包
**数据储存形式**：dataset

对数据统一处理，可以像访问数组元素一样访问数据集中的元素

支持索引下标，其中提取出来的元素为features，target格式

**加载器DataLoader**

负责在程序中对数据集的使用，如逐批加载数据

**采样器sampler**

提供了一个每一批抽取数据集中样本的方法

In [6]:
# idx=100
# muteimg=train_dataset[idx][0].numpy()
# plt.imshow(muteimg[0,...])
# print('标签是： ',train_dataset[idx][1])
#图像是一个灰度矩阵，用imshow画图会将灰度矩阵自动展现为彩色

## 构建网络
1. 构造ConvNet类，继承nn.Module父类
2. 重写_init_()构造函数和forward()函数

In [13]:
depth=[4,8]
#定义卷积神经网络，4和8为人为指定的两个卷积层的厚度
class ConvNet(nn.Module):
    def __init__(self):
        #该函数在创建一个ConvNet对象即调用net=ConvNet()时会被调用
        #首先调用父类相应的构造函数
        super(ConvNet,self).__init__()
        
        #其次构造ConvNet需要用到的各个模块，此处定义组建并非真正搭建组件
        #定义一个卷积层
        self.conv1=nn.Conv2d(1,4,5,padding=2) #输入通道为1，输出通道为4，窗口大小为5
        self.pool=nn.MaxPool2d(2,2) #进行窗口为2*2的池化运算
        #第二层卷积
        self.conv2=nn.Conv2d(depth[0],depth[1],5,padding=2)
        #线性连接层，输入尺寸为最后一层立方体的线性平铺，输出层为512个节点
        self.fc1=nn.Linear(image_size//4*image_size//4*depth[1],512) #//为取整除
        self.fc2=nn.Linear(512,num_classes)#最后一层线性分类单元
        
    def forward(self,x): #该函数完成前向运算，是真正神经网络的拼装
        x=self.conv1(x)
        #x的尺寸：（batch_size,image_channels,image_with,image_hight)
        x=F.relu(x) #激活函数
        #x的尺寸：(batch_size,num_filters,image_with,image_height)
        
        x=self.pool(x)
        #x的尺寸：(batch_size,depth[0],image_width/2,image_hight/2)
        
        x=self.conv2(x)
        x=F.relu(x)
        #x的尺寸：(batch_size,depth[1],image_width/2,image_hight/2)
        
        x=self.pool(x)
        #x的尺寸：(batch_size,depth[1],image_width/4,image_hight/4)
        
        #将立体特征图转化为一个一维向量
        x=x.view(-1,image_size//4*image_size//4*depth[1])
        
        #第五层为全连接层，使用ReLU函数
        x=F.relu(self.fc1(x)) #此时x的尺寸为(batch_size,512)
        
        #以0.5的概率对这一层进行dropout操作，防止过拟合
        x=F.dropout(x,training=self.training)
        
        x=self.fc2(x)
        x=F.log_softmax(x,dim=1)
        return x
    
    def retrieve_features(self,x): #该函数用于提取卷积神经网络的特征图
        feature_map1=F.relu(self.conv1(x))
        x=self.pool(feature_map1)
        feature_map2=F.relu(self.conv2(x))
        return (feature_map1,feature_map2)

drop out 可以提升模型的泛化能力

In [17]:
def rightness(predictions,labels):
    pred=torch.max(predictions.data,1)[1]
    rights= pred.eq(labels.data.view_as(pred)).sum()
    return rights,len(labels)

## 运行模型

In [23]:
net = ConvNet()

criterion=nn.CrossEntropyLoss() #Loss 的定义，交叉熵
optimizer=optim.SGD(net.parameters(),lr=0.001,momentum=0.9) #定义优化器

record=[]
weights=[]

for epoch in range(num_epochs):
    train_rights=[] #记录训练集上的准确率
    
    for batch_idx,(data,target) in enumerate(train_loader):
        data, target=data.clone().requires_grad_(True), target.clone().detach() #tensor不向前传播
        net.train() #给网络做标记，标志模型在训练集上训练，此时打开dropout
        
        output=net(data) #神经网络完成一次前馈计算，得到预测的output
        loss=criterion(output,target) #计算误差
        optimizer.zero_grad()
        loss.backward()
        optimizer.step() #一步随机梯度优化算法
        right=rightness(output,target)
        train_rights.append(right)
        
        if batch_idx % 100==0:
            net.eval() #给网络做标记，标志模型在校验集上训练，此时关闭dropout
            val_rights=[]
            for (data,target) in validation_loader:
                data,target= data.clone().requires_grad_(True), target.clone().detach()
                output=net(data)
                right=rightness(output,target)
                val_rights.append(right)
                
                train_r=(sum([tup[0] for tup in train_rights]),sum([tup[1] for tup in train_rights]))
                #二元组：分别记录训练集中分类正确的数量和总的样本数
                val_r=(sum([tup[0] for tup in val_rights]),sum([tup[1] for tup in val_rights]))
                print('训练周期：{}[{}/{} ({:.0f}%)]\t,Loss:{:.6f}\t,训练准确率：{:.2f}%\t,校验准确率：{:.2f}%'.format(
                    epoch,batch_idx*len(data),len(train_loader.dataset),
                    100.*batch_idx/len(train_loader),loss.data,
                    100.*train_r[0]/train_r[1],
                    100.*val_r[0]/val_r[1]))
                record.append((100-100.*train_r[0]/train_r[1],100-100.*val_r[0]/val_r[1])) #记录下错误率
                weights.append([net.conv1.weight.data.clone(),net.conv1.bias.data.clone(),
                               net.conv2.weight.data.clone(),net.conv2.bias.data.clone()]) 
                #clone将数据备份，否则当weight.data变化时，列表中的每一项数据也会联动
                
    

训练周期：0[0/60000 (0%)]	,Loss:2.305213	,训练准确率：9.38%	,校验准确率：10.94%
训练周期：0[0/60000 (0%)]	,Loss:2.305213	,训练准确率：9.38%	,校验准确率：15.62%
训练周期：0[0/60000 (0%)]	,Loss:2.305213	,训练准确率：9.38%	,校验准确率：14.06%
训练周期：0[0/60000 (0%)]	,Loss:2.305213	,训练准确率：9.38%	,校验准确率：12.89%
训练周期：0[0/60000 (0%)]	,Loss:2.305213	,训练准确率：9.38%	,校验准确率：11.56%
训练周期：0[0/60000 (0%)]	,Loss:2.305213	,训练准确率：9.38%	,校验准确率：10.94%
训练周期：0[0/60000 (0%)]	,Loss:2.305213	,训练准确率：9.38%	,校验准确率：11.16%
训练周期：0[0/60000 (0%)]	,Loss:2.305213	,训练准确率：9.38%	,校验准确率：11.13%
训练周期：0[0/60000 (0%)]	,Loss:2.305213	,训练准确率：9.38%	,校验准确率：11.28%
训练周期：0[0/60000 (0%)]	,Loss:2.305213	,训练准确率：9.38%	,校验准确率：12.34%
训练周期：0[0/60000 (0%)]	,Loss:2.305213	,训练准确率：9.38%	,校验准确率：12.22%
训练周期：0[0/60000 (0%)]	,Loss:2.305213	,训练准确率：9.38%	,校验准确率：12.37%
训练周期：0[0/60000 (0%)]	,Loss:2.305213	,训练准确率：9.38%	,校验准确率：12.50%
训练周期：0[0/60000 (0%)]	,Loss:2.305213	,训练准确率：9.38%	,校验准确率：12.61%
训练周期：0[0/60000 (0%)]	,Loss:2.305213	,训练准确率：9.38%	,校验准确率：12.29%
训练周期：0[0/60000 (0%)]	,Loss:2.305213	,训练准确率：9.38%	,校验准确率

训练周期：0[6400/60000 (11%)]	,Loss:2.294006	,训练准确率：11.19%	,校验准确率：18.47%
训练周期：0[6400/60000 (11%)]	,Loss:2.294006	,训练准确率：11.19%	,校验准确率：18.49%
训练周期：0[6400/60000 (11%)]	,Loss:2.294006	,训练准确率：11.19%	,校验准确率：18.56%
训练周期：0[6400/60000 (11%)]	,Loss:2.294006	,训练准确率：11.19%	,校验准确率：18.44%
训练周期：0[6400/60000 (11%)]	,Loss:2.294006	,训练准确率：11.19%	,校验准确率：18.42%
训练周期：0[6400/60000 (11%)]	,Loss:2.294006	,训练准确率：11.19%	,校验准确率：18.38%
训练周期：0[6400/60000 (11%)]	,Loss:2.294006	,训练准确率：11.19%	,校验准确率：18.39%
训练周期：0[800/60000 (11%)]	,Loss:2.294006	,训练准确率：11.19%	,校验准确率：18.44%
训练周期：0[12800/60000 (21%)]	,Loss:2.282098	,训练准确率：12.85%	,校验准确率：20.31%
训练周期：0[12800/60000 (21%)]	,Loss:2.282098	,训练准确率：12.85%	,校验准确率：19.53%
训练周期：0[12800/60000 (21%)]	,Loss:2.282098	,训练准确率：12.85%	,校验准确率：18.23%
训练周期：0[12800/60000 (21%)]	,Loss:2.282098	,训练准确率：12.85%	,校验准确率：17.58%
训练周期：0[12800/60000 (21%)]	,Loss:2.282098	,训练准确率：12.85%	,校验准确率：18.44%
训练周期：0[12800/60000 (21%)]	,Loss:2.282098	,训练准确率：12.85%	,校验准确率：20.05%
训练周期：0[12800/60000 (21%)]	,Loss:2.282098	,训

训练周期：0[19200/60000 (32%)]	,Loss:2.258200	,训练准确率：15.81%	,校验准确率：36.76%
训练周期：0[19200/60000 (32%)]	,Loss:2.258200	,训练准确率：15.81%	,校验准确率：36.83%
训练周期：0[19200/60000 (32%)]	,Loss:2.258200	,训练准确率：15.81%	,校验准确率：36.76%
训练周期：0[19200/60000 (32%)]	,Loss:2.258200	,训练准确率：15.81%	,校验准确率：36.66%
训练周期：0[19200/60000 (32%)]	,Loss:2.258200	,训练准确率：15.81%	,校验准确率：36.76%
训练周期：0[19200/60000 (32%)]	,Loss:2.258200	,训练准确率：15.81%	,校验准确率：36.82%
训练周期：0[19200/60000 (32%)]	,Loss:2.258200	,训练准确率：15.81%	,校验准确率：36.95%
训练周期：0[19200/60000 (32%)]	,Loss:2.258200	,训练准确率：15.81%	,校验准确率：37.08%
训练周期：0[19200/60000 (32%)]	,Loss:2.258200	,训练准确率：15.81%	,校验准确率：37.13%
训练周期：0[19200/60000 (32%)]	,Loss:2.258200	,训练准确率：15.81%	,校验准确率：37.39%
训练周期：0[19200/60000 (32%)]	,Loss:2.258200	,训练准确率：15.81%	,校验准确率：37.46%
训练周期：0[19200/60000 (32%)]	,Loss:2.258200	,训练准确率：15.81%	,校验准确率：37.60%
训练周期：0[19200/60000 (32%)]	,Loss:2.258200	,训练准确率：15.81%	,校验准确率：37.64%
训练周期：0[19200/60000 (32%)]	,Loss:2.258200	,训练准确率：15.81%	,校验准确率：37.70%
训练周期：0[19200/60000 (32%)]	,Loss:2.

训练周期：0[32000/60000 (53%)]	,Loss:1.630103	,训练准确率：23.84%	,校验准确率：62.50%
训练周期：0[32000/60000 (53%)]	,Loss:1.630103	,训练准确率：23.84%	,校验准确率：67.97%
训练周期：0[32000/60000 (53%)]	,Loss:1.630103	,训练准确率：23.84%	,校验准确率：68.23%
训练周期：0[32000/60000 (53%)]	,Loss:1.630103	,训练准确率：23.84%	,校验准确率：69.53%
训练周期：0[32000/60000 (53%)]	,Loss:1.630103	,训练准确率：23.84%	,校验准确率：68.75%
训练周期：0[32000/60000 (53%)]	,Loss:1.630103	,训练准确率：23.84%	,校验准确率：66.93%
训练周期：0[32000/60000 (53%)]	,Loss:1.630103	,训练准确率：23.84%	,校验准确率：65.40%
训练周期：0[32000/60000 (53%)]	,Loss:1.630103	,训练准确率：23.84%	,校验准确率：64.84%
训练周期：0[32000/60000 (53%)]	,Loss:1.630103	,训练准确率：23.84%	,校验准确率：64.58%
训练周期：0[32000/60000 (53%)]	,Loss:1.630103	,训练准确率：23.84%	,校验准确率：64.53%
训练周期：0[32000/60000 (53%)]	,Loss:1.630103	,训练准确率：23.84%	,校验准确率：64.77%
训练周期：0[32000/60000 (53%)]	,Loss:1.630103	,训练准确率：23.84%	,校验准确率：64.97%
训练周期：0[32000/60000 (53%)]	,Loss:1.630103	,训练准确率：23.84%	,校验准确率：64.30%
训练周期：0[32000/60000 (53%)]	,Loss:1.630103	,训练准确率：23.84%	,校验准确率：63.50%
训练周期：0[32000/60000 (53%)]	,Loss:1.

训练周期：0[38400/60000 (64%)]	,Loss:0.800863	,训练准确率：30.67%	,校验准确率：74.67%
训练周期：0[38400/60000 (64%)]	,Loss:0.800863	,训练准确率：30.67%	,校验准确率：74.77%
训练周期：0[38400/60000 (64%)]	,Loss:0.800863	,训练准确率：30.67%	,校验准确率：74.65%
训练周期：0[38400/60000 (64%)]	,Loss:0.800863	,训练准确率：30.67%	,校验准确率：74.80%
训练周期：0[38400/60000 (64%)]	,Loss:0.800863	,训练准确率：30.67%	,校验准确率：74.95%
训练周期：0[38400/60000 (64%)]	,Loss:0.800863	,训练准确率：30.67%	,校验准确率：75.02%
训练周期：0[38400/60000 (64%)]	,Loss:0.800863	,训练准确率：30.67%	,校验准确率：75.02%
训练周期：0[38400/60000 (64%)]	,Loss:0.800863	,训练准确率：30.67%	,校验准确率：75.05%
训练周期：0[38400/60000 (64%)]	,Loss:0.800863	,训练准确率：30.67%	,校验准确率：75.00%
训练周期：0[38400/60000 (64%)]	,Loss:0.800863	,训练准确率：30.67%	,校验准确率：75.13%
训练周期：0[38400/60000 (64%)]	,Loss:0.800863	,训练准确率：30.67%	,校验准确率：75.20%
训练周期：0[38400/60000 (64%)]	,Loss:0.800863	,训练准确率：30.67%	,校验准确率：75.07%
训练周期：0[38400/60000 (64%)]	,Loss:0.800863	,训练准确率：30.67%	,校验准确率：75.06%
训练周期：0[38400/60000 (64%)]	,Loss:0.800863	,训练准确率：30.67%	,校验准确率：75.15%
训练周期：0[38400/60000 (64%)]	,Loss:0.

...

训练周期：19[12800/60000 (21%)]	,Loss:0.042418	,训练准确率：98.06%	,校验准确率：98.26%
训练周期：19[12800/60000 (21%)]	,Loss:0.042418	,训练准确率：98.06%	,校验准确率：98.16%
训练周期：19[12800/60000 (21%)]	,Loss:0.042418	,训练准确率：98.06%	,校验准确率：98.11%
训练周期：19[12800/60000 (21%)]	,Loss:0.042418	,训练准确率：98.06%	,校验准确率：98.02%
训练周期：19[12800/60000 (21%)]	,Loss:0.042418	,训练准确率：98.06%	,校验准确率：97.98%
训练周期：19[12800/60000 (21%)]	,Loss:0.042418	,训练准确率：98.06%	,校验准确率：97.95%
训练周期：19[12800/60000 (21%)]	,Loss:0.042418	,训练准确率：98.06%	,校验准确率：97.92%
训练周期：19[12800/60000 (21%)]	,Loss:0.042418	,训练准确率：98.06%	,校验准确率：97.98%
训练周期：19[12800/60000 (21%)]	,Loss:0.042418	,训练准确率：98.06%	,校验准确率：97.95%
训练周期：19[12800/60000 (21%)]	,Loss:0.042418	,训练准确率：98.06%	,校验准确率：97.96%
训练周期：19[12800/60000 (21%)]	,Loss:0.042418	,训练准确率：98.06%	,校验准确率：97.97%
训练周期：19[12800/60000 (21%)]	,Loss:0.042418	,训练准确率：98.06%	,校验准确率：97.99%
训练周期：19[12800/60000 (21%)]	,Loss:0.042418	,训练准确率：98.06%	,校验准确率：98.00%
训练周期：19[12800/60000 (21%)]	,Loss:0.042418	,训练准确率：98.06%	,校验准确率：97.93%
训练周期：19[12800/60000 

训练周期：19[25600/60000 (43%)]	,Loss:0.107799	,训练准确率：98.17%	,校验准确率：100.00%
训练周期：19[25600/60000 (43%)]	,Loss:0.107799	,训练准确率：98.17%	,校验准确率：97.66%
训练周期：19[25600/60000 (43%)]	,Loss:0.107799	,训练准确率：98.17%	,校验准确率：97.92%
训练周期：19[25600/60000 (43%)]	,Loss:0.107799	,训练准确率：98.17%	,校验准确率：97.66%
训练周期：19[25600/60000 (43%)]	,Loss:0.107799	,训练准确率：98.17%	,校验准确率：97.81%
训练周期：19[25600/60000 (43%)]	,Loss:0.107799	,训练准确率：98.17%	,校验准确率：98.18%
训练周期：19[25600/60000 (43%)]	,Loss:0.107799	,训练准确率：98.17%	,校验准确率：98.44%
训练周期：19[25600/60000 (43%)]	,Loss:0.107799	,训练准确率：98.17%	,校验准确率：98.24%
训练周期：19[25600/60000 (43%)]	,Loss:0.107799	,训练准确率：98.17%	,校验准确率：97.92%
训练周期：19[25600/60000 (43%)]	,Loss:0.107799	,训练准确率：98.17%	,校验准确率：98.12%
训练周期：19[25600/60000 (43%)]	,Loss:0.107799	,训练准确率：98.17%	,校验准确率：98.30%
训练周期：19[25600/60000 (43%)]	,Loss:0.107799	,训练准确率：98.17%	,校验准确率：98.18%
训练周期：19[25600/60000 (43%)]	,Loss:0.107799	,训练准确率：98.17%	,校验准确率：98.20%
训练周期：19[25600/60000 (43%)]	,Loss:0.107799	,训练准确率：98.17%	,校验准确率：98.21%
训练周期：19[25600/60000

训练周期：19[32000/60000 (53%)]	,Loss:0.138867	,训练准确率：98.11%	,校验准确率：98.10%
训练周期：19[32000/60000 (53%)]	,Loss:0.138867	,训练准确率：98.11%	,校验准确率：98.14%
训练周期：19[32000/60000 (53%)]	,Loss:0.138867	,训练准确率：98.11%	,校验准确率：98.14%
训练周期：19[32000/60000 (53%)]	,Loss:0.138867	,训练准确率：98.11%	,校验准确率：98.09%
训练周期：19[32000/60000 (53%)]	,Loss:0.138867	,训练准确率：98.11%	,校验准确率：98.10%
训练周期：19[32000/60000 (53%)]	,Loss:0.138867	,训练准确率：98.11%	,校验准确率：98.10%
训练周期：19[32000/60000 (53%)]	,Loss:0.138867	,训练准确率：98.11%	,校验准确率：98.05%
训练周期：19[32000/60000 (53%)]	,Loss:0.138867	,训练准确率：98.11%	,校验准确率：98.06%
训练周期：19[32000/60000 (53%)]	,Loss:0.138867	,训练准确率：98.11%	,校验准确率：98.01%
训练周期：19[32000/60000 (53%)]	,Loss:0.138867	,训练准确率：98.11%	,校验准确率：98.02%
训练周期：19[32000/60000 (53%)]	,Loss:0.138867	,训练准确率：98.11%	,校验准确率：98.05%
训练周期：19[32000/60000 (53%)]	,Loss:0.138867	,训练准确率：98.11%	,校验准确率：98.08%
训练周期：19[32000/60000 (53%)]	,Loss:0.138867	,训练准确率：98.11%	,校验准确率：98.07%
训练周期：19[32000/60000 (53%)]	,Loss:0.138867	,训练准确率：98.11%	,校验准确率：98.07%
训练周期：19[32000/60000 

训练周期：19[44800/60000 (75%)]	,Loss:0.023995	,训练准确率：98.12%	,校验准确率：98.12%
训练周期：19[44800/60000 (75%)]	,Loss:0.023995	,训练准确率：98.12%	,校验准确率：98.08%
训练周期：19[44800/60000 (75%)]	,Loss:0.023995	,训练准确率：98.12%	,校验准确率：98.15%
训练周期：19[44800/60000 (75%)]	,Loss:0.023995	,训练准确率：98.12%	,校验准确率：98.10%
训练周期：19[44800/60000 (75%)]	,Loss:0.023995	,训练准确率：98.12%	,校验准确率：98.11%
训练周期：19[44800/60000 (75%)]	,Loss:0.023995	,训练准确率：98.12%	,校验准确率：98.02%
训练周期：19[44800/60000 (75%)]	,Loss:0.023995	,训练准确率：98.12%	,校验准确率：97.93%
训练周期：19[44800/60000 (75%)]	,Loss:0.023995	,训练准确率：98.12%	,校验准确率：97.95%
训练周期：19[44800/60000 (75%)]	,Loss:0.023995	,训练准确率：98.12%	,校验准确率：98.01%
训练周期：19[44800/60000 (75%)]	,Loss:0.023995	,训练准确率：98.12%	,校验准确率：97.98%
训练周期：19[44800/60000 (75%)]	,Loss:0.023995	,训练准确率：98.12%	,校验准确率：97.99%
训练周期：19[44800/60000 (75%)]	,Loss:0.023995	,训练准确率：98.12%	,校验准确率：98.00%
训练周期：19[44800/60000 (75%)]	,Loss:0.023995	,训练准确率：98.12%	,校验准确率：97.97%
训练周期：19[44800/60000 (75%)]	,Loss:0.023995	,训练准确率：98.12%	,校验准确率：98.03%
训练周期：19[44800/60000 

训练周期：19[51200/60000 (85%)]	,Loss:0.028812	,训练准确率：98.09%	,校验准确率：97.99%
训练周期：19[51200/60000 (85%)]	,Loss:0.028812	,训练准确率：98.09%	,校验准确率：97.99%
训练周期：19[51200/60000 (85%)]	,Loss:0.028812	,训练准确率：98.09%	,校验准确率：97.98%
训练周期：19[51200/60000 (85%)]	,Loss:0.028812	,训练准确率：98.09%	,校验准确率：97.99%
训练周期：19[51200/60000 (85%)]	,Loss:0.028812	,训练准确率：98.09%	,校验准确率：97.97%
训练周期：19[51200/60000 (85%)]	,Loss:0.028812	,训练准确率：98.09%	,校验准确率：98.00%
训练周期：19[6400/60000 (85%)]	,Loss:0.028812	,训练准确率：98.09%	,校验准确率：98.00%
训练周期：19[57600/60000 (96%)]	,Loss:0.021102	,训练准确率：98.11%	,校验准确率：100.00%
训练周期：19[57600/60000 (96%)]	,Loss:0.021102	,训练准确率：98.11%	,校验准确率：98.44%
训练周期：19[57600/60000 (96%)]	,Loss:0.021102	,训练准确率：98.11%	,校验准确率：98.96%
训练周期：19[57600/60000 (96%)]	,Loss:0.021102	,训练准确率：98.11%	,校验准确率：98.44%
训练周期：19[57600/60000 (96%)]	,Loss:0.021102	,训练准确率：98.11%	,校验准确率：98.44%
训练周期：19[57600/60000 (96%)]	,Loss:0.021102	,训练准确率：98.11%	,校验准确率：98.70%
训练周期：19[57600/60000 (96%)]	,Loss:0.021102	,训练准确率：98.11%	,校验准确率：98.66%
训练周期：19[57600/60000 

In [24]:
net.eval()
vals=[]
for data,target in test_loader:
    data,target=data.clone().detach().requires_grad_(True),target.clone().detach()
    output=net(data)
    val=rightness(output,target)
    vals.append(val)
    
    #计算准确率
    rights=(sum([tup[0] for tup in vals]),sum([tup[1] for tup in vals]))
    right_rate=1.0*rights[0]/rights[1]
    right_rate
    
#     plt.figure(figsize=(10,7))
#     plt.plot(record)
#     plt.xlabel('steps')
#     plt.ylabel('error rate')

In [25]:
right_rate

tensor(0.9914)

tensor.detach（）

返回一个新的tensor，从当前计算图中分离下来的，但是仍指向原变量的存放位置,不同之处只是requires_grad为false，得到的这个tensor永远不需要计算其梯度，不具有grad。

## 剖析卷积神经网络（略）